## Carga del dataframe limpio

In [2]:
import pandas as pd

# Cargar el archivo Excel limpio
file_path = "../data/processed/GSAF5_cleaned.xlsx"
df = pd.read_excel(file_path, engine="openpyxl")

# Mostrar las primeras filas del DataFrame
df.head()

,date,year,type,country,state,location,activity,name,sex,age,...,species,source,pdf,href_formula,href,case_number,case_number.1,original_order,unnamed:_21,unnamed:_22
0,2025-02-07 00:00:00,2025.0,Unprovoked,Turks and Caicos,NaN,Thompson Cove Beach,Swimming,Unknown,F,55,...,Unknown,Todd Smith: Platform X,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,25-Sep-2022,2022.0,Unprovoked,SOUTH AFRICA,Western Cape Province,"Central Beach, Plettenberg Bay",Swimming,Kimon Bisogno,F,39,...,"White shark, 13'","Mirror, 9/25/2022",2022.09.25-Plett.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2022.09.25,2022.09.25,6802.0,NaN,NaN
2,06-Sep-2022,2022.0,Unprovoked,BAHAMAS,NaN,Green Cay,Snorkeling,Caroline DiPlacido,F,58,...,Bull shark,"B. Myatt, GSAF",2022.09.06-Bahamas.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2022.09.06,2022.09.06,6801.0,NaN,NaN
3,03-Sep-2022,2022.0,Unprovoked,USA,Hawaii,"Lower Paia Beach Park, Maui",Swimming or Snorkeling,female,F,51,...,NaN,"Star Advertiser, 9/3/2022",2022.09.03-Maui.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2022.09.03,2022.09.03,6800.0,NaN,NaN
4,31-Aug-2022,2022.0,Unprovoked,AUSTRALIA,New South Wales,Avoca,Surfing,Sunni Pace,M,14,...,Bronze whaler,"Surfline, 9/2/2022",2022.08.31-Pace.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2022.08.31,2022.08.31,6799.0,NaN,NaN


## Países con más ataques

In [6]:
ataques_pais = df.groupby('country').size().sort_values(ascending=False)
ataques_pais   

country
USA                 2475
AUSTRALIA           1449
SOUTH AFRICA         591
FRANCE               176
PAPUA NEW GUINEA     153
                    ... 
GABON                  1
SLOVENIA               1
DOMINIKANA             1
DJIBOUTI               1
AFRICA                 1
Length: 152, dtype: int64

## Países seguros

In [7]:
umbral = 5
paises_seguros = ataques_pais[ataques_pais < umbral]  # umbral podría ser 5, por ejemplo
paises_seguros


country
MICRONESIA              4
EL SALVADOR             4
NIGERIA                 4
GRENADA                 4
UNITED ARAB EMIRATES    4
                       ..
GABON                   1
SLOVENIA                1
DOMINIKANA              1
DJIBOUTI                1
AFRICA                  1
Length: 81, dtype: int64